# Transform Learning

**This notebook contains uses of transfer learning to create better model for custom dataset.**

### Imports

In [65]:
import sys
sys.path.insert(0, '../scripts')

import matplotlib.pyplot as plt
import torch
import torchvision
from torchinfo import summary
from scripts.data_setup import create_dataloaders
from pathlib import Path
import zipfile

### Device agnostic code

In [66]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

### Getting data

In [67]:
data_name = 'baklava_churros_cheesecake'
data_path = Path('../data/processed')
images_path = data_path / data_name
zip_path = Path(str(images_path) + '.zip')

if not images_path.is_dir():
    images_path.mkdir(parents=True, exist_ok=True)
    if zip_path.exists():
        with zipfile.ZipFile(zip_path, 'r') as zip:
            zip.extractall(images_path)
    else:
        print(f'{data_name} is wrong name or custom dataset was not created.\n'
              'Use custom dataset notebook to create dataset.')
        images_path.rmdir()

train_dir = images_path / 'train'
test_dir = images_path / 'test'

### Creating DataLoaders

In [68]:
weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT
transforms = weights.transforms()

train_dataloader, test_dataloader, class_names = create_dataloaders(train_dir=train_dir,
                                                                    test_dir=test_dir,
                                                                    transform=transforms,
                                                                    batch_size=32)